In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import h5py
import os
import glob
import warnings
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

from utils.plot_utils import *
np.set_printoptions(precision=3, suppress=True)

warnings.filterwarnings('ignore')

# Learning a model for myosin

In [13]:
from utils.library.composite_library import fill_group_info, collect_library, take_time_derivatives, decompose_library
from utils.library.extra_functions import *
        
libraries = [
    ('m_ij', 'Public/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP', 'tensor_library', 'tensor_SVDPipeline_PMG_CF.pkl'),
    ('v', 'Public/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP', 'tensor_library', None), #Not focused on flow dynamics
    ('c', 'Public/WT/Moesin-GFP', 'scalar_library', None), #Not focused on cadherin dynamics, just include it as a source
]

    
extra_functions = [
    lambda data: material_derivative_terms(data, key='m_ij'),
    lambda data: add_static_sources(data, couple='m_ij'),
    lambda data: symmetric_tensor_couple(data, ['m_ij', 'E']),
    lambda data: multiply_tensor_by_scalar(data, None, ['c']),
]


with h5py.File(os.path.join(libraries[0][1], 'derivative_library.h5'), 'r') as h5f:
    embryoIDs = list(h5f.keys())

with h5py.File('Public/myosin_fitting_with_actin.h5', 'w') as data:
    with tqdm(total=len(embryoIDs)) as pbar:
        pbar.set_description('Collecting embryo data')
        for embryoID in embryoIDs:
            pbar.set_postfix(embryoID=embryoID)
            group, feature_names = fill_group_info(data, embryoID, libraries)
            collect_library(group, 
                            libraries, 
                            feature_names, 
                            extra_functions=extra_functions)
            take_time_derivatives(group, libraries, window_length=7)
            decompose_library(group, libraries)
            
            pbar.update()

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
from utils.modeling.sindy_utils import *
from utils.modeling.fly_sindy import FlySINDy
import pysindy as ps

with h5py.File('Public/myosin_fitting_with_actin.h5', 'r') as h5f:
    X, X_dot, feature_names = collect_library_terms(h5f, 'm_ij', tmin=-5, tmax=10, feature_names=None)

sindy = FlySINDy(
    optimizer=ps.STLSQ(threshold=1e-2, alpha=1e1, normalize_columns=True),
    feature_names = feature_names,
    n_models=10,
    n_candidates_to_drop=0,
    subset_fraction=0.1)
sindy.fit(x=X, x_dot=X_dot)
print(f'\nThreshold = {1e-2:g}, Alpha = {1e1:g}')
sindy.print(lhs=['D_t m_ij'])

  0%|          | 0/7 [00:00<?, ?it/s]


Threshold = 0.01, Alpha = 10
D_t m_ij = (0.099 + -0.077 c) Static_DV Tr(m_ij) + (-0.229 + 0.214 c) m_ij + (0.382 + 0.433 c) m_ij Tr(E) + (1.630 + -1.460 c) m_ij Tr(m_ij)


In [19]:
print(len(sindy.feature_names))
print(sindy.feature_names)

16
['E', 'E Tr(m_ij)', 'Static_DV', 'Static_DV Tr(m_ij)', 'c E', 'c E Tr(m_ij)', 'c Static_DV', 'c Static_DV Tr(m_ij)', 'c m_ij', 'c m_ij Tr(E)', 'c m_ij Tr(m_ij)', 'c {m_ij, E}', 'm_ij', 'm_ij Tr(E)', 'm_ij Tr(m_ij)', '{m_ij, E}']


In [16]:
'''
Repeat the fitting process restricted to the library terms above
We do this because ensembling suppresses coefficients due to subleading terms!
'''
from utils.modeling.sindy_utils import *
from utils.modeling.fly_sindy import FlySINDy
import pysindy as ps

with h5py.File('Public/myosin_fitting_with_actin.h5', 'r') as h5f:
    X, X_dot, feature_names = collect_library_terms(h5f, 'm_ij', tmin=-5, tmax=10, feature_names=overleaf_feature_names)


#No longer threshold since we've established the above terms
sindy = FlySINDy(
    optimizer=ps.STLSQ(threshold=0, alpha=0, normalize_columns=True),
    feature_names = feature_names,
    n_models=1,
    subset_fraction=0.1)
sindy.fit(x=X, x_dot=X_dot)
print(f'\nThreshold = {0:g}, Alpha = {0:g}')
sindy.print(lhs=['D_t m_ij'])

  0%|          | 0/7 [00:00<?, ?it/s]


Threshold = 0, Alpha = 0
D_t m_ij = (-0.209 + 0.194 c) m_ij + (1.431 + -1.253 c) m_ij Tr(m_ij) + (0.463 + 0.358 c) m_ij Tr(E) + (0.095 + -0.075 c) Static_DV Tr(m_ij)


# Learning a model for cadherin

In [16]:
from utils.library.composite_library import fill_group_info, collect_library, take_time_derivatives, decompose_library
from utils.library.extra_functions import *
        
libraries = [
    ('c', 'Public/WT/ECad-GFP', 'scalar_library', 'raw_SVDPipeline_PMG_CF.pkl'), #Want cadherin dynamics
    ('m_ij', 'Public/Halo_Hetero_Twist[ey53]_Hetero/Sqh-GFP', 'tensor_library', None), #Not focused on myosin dynamics
    ('v', 'Public/WT/ECad-GFP', 'tensor_library', None), #Not focused on flow dynamics
]

    
extra_functions = [
    lambda data: material_derivative_terms(data, key='c'),
    lambda data: add_v_squared(data),
    lambda data: tensor_trace(data, keys=['m_ij', 'E']),
    lambda data: scalar_couple(data, keys=['c', 'Tr(E)', 'Tr(m_ij)']),
    lambda data: delete_high_order_scalars(data, max_space_order=1),
]


with h5py.File(os.path.join(libraries[0][1], 'derivative_library.h5'), 'r') as h5f:
    embryoIDs = list(h5f.keys())

with h5py.File('Public/test_cadherin_fitting.h5', 'w') as data:
    with tqdm(total=len(embryoIDs)) as pbar:
        pbar.set_description('Collecting embryo data')
        for embryoID in embryoIDs:
            pbar.set_postfix(embryoID=embryoID)
            group, feature_names = fill_group_info(data, embryoID, libraries)
            collect_library(group, 
                            libraries, 
                            feature_names, 
                            extra_functions=extra_functions)
            take_time_derivatives(group, libraries, window_length=7)
            decompose_library(group, libraries)
            
            pbar.update()

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
from utils.modeling.sindy_utils import *
from utils.modeling.fly_sindy import FlySINDy
import pysindy as ps

with h5py.File('Public/test_cadherin_fitting.h5', 'r') as h5f:
    X, X_dot, feature_names = collect_library_terms(h5f, 'c', tmin=-5, tmax=10, feature_names=None)

sindy = FlySINDy(
    optimizer=ps.STLSQ(threshold=1e-2, alpha=1e1, normalize_columns=True),
    feature_names = feature_names,
    n_models=10,
    n_candidates_to_drop=0,
    subset_fraction=0.1)
sindy.fit(x=X, x_dot=X_dot)
print(f'\nThreshold = {1e-2:g}, Alpha = {1e1:g}')
sindy.print(lhs=['D_t c'])

  0%|          | 0/7 [00:00<?, ?it/s]


Threshold = 0.01, Alpha = 10
D_t c = 0
